In [1]:
from Env import EnvLabyrinthe
import numpy as np
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

In [26]:
env = EnvLabyrinthe(
    nblignes = 3,
    nbcolonnes = 3,
    alea = 1, # 0 : deterministe / 1 : stochastique
    proba_mur = 0.15,
    proba_blanc = 0.45,
    proba_vert = 0,
    proba_bleue = 0.2,
    proba_rouge = 0.2
)

env.render()

J 0 0 

0 0 0 

0 0 0 



In [27]:
MDPbi = env.getMDPBiObj()
print(MDPbi)

{0: [[(0, 1.0, (0, 0), False)], [(0, 1.0, (0, 0), False)], [(7, 0.6875, (-1, -1), False), (3, 0.0625, (-1, -1), False), (4, 0.0625, (-1, -1), False), (5, 0.0625, (-1, -1), False), (6, 0.0625, (-1, -1), False), (8, 0.0625, (10000000000.0, 10000000000.0), True)], [(5, 0.6875, (-1, -1), False), (1, 0.0625, (-1, -1), False), (2, 0.0625, (-1, -1), False), (4, 0.0625, (-1, -1), False), (7, 0.0625, (-1, -1), False), (8, 0.0625, (10000000000.0, 10000000000.0), True)], [(0, 1.0, (0, 0), False)], [(0, 1.0, (0, 0), False)], [(0, 1.0, (0, 0), False)], [(0, 1.0, (0, 0), False)]], 1: [[(1, 1.0, (0, 0), False)], [(6, 0.8125, (-1, -1), False), (3, 0.0625, (-1, -1), False), (4, 0.0625, (-1, -1), False), (7, 0.0625, (-1, -1), False)], [(8, 0.8125, (10000000000.0, 10000000000.0), True), (4, 0.0625, (-1, -1), False), (5, 0.0625, (-1, -1), False), (7, 0.0625, (-1, -1), False)], [(1, 1.0, (0, 0), False)], [(1, 1.0, (0, 0), False)], [(1, 1.0, (0, 0), False)], [(1, 1.0, (0, 0), False)], [(1, 1.0, (0, 0), Fals

In [28]:
#len(MDPbi)
env.nb_states

9

In [29]:
discount_factor = 0.5

def get_pred_by_action(MDPbi,state,action) :
     
    list_pred_by_action = list()    
    
    for pred in range(len(MDPbi)) :
        
        for choice in MDPbi[pred][action] :
            
            if choice[0] == state : # si le prochain état = state
            
                list_pred_by_action.append((choice,pred))
                
    return list_pred_by_action


def get_first_member_matrix_and_obj(MDPbi,discount_factor,nb_obs,nb_actions,infos_state_dict) :
    
    objective_matrix = np.zeros((2,nb_obs,nb_actions))
    # Autant de contraintes que d'état
    first_member_matrix = np.zeros((nb_obs,nb_obs,nb_actions)) # (state,pred,action)
    
    
    for state in range(nb_obs) :
        
        first_member_matrix[state,state,:] = 1
        
        # first_member_matrix[state,action] += 
        for action in range(len(MDPbi[state])) :
            
            
            if infos_state_dict[state] == 2 : # bleu
                objective_matrix[0,state,action] = 2
                objective_matrix[1,state,action] = 0
            elif infos_state_dict[state] == 3 : # rouge
                objective_matrix[0,state,action] = 0
                objective_matrix[1,state,action] = 2
            else :
                objective_matrix[0,state,action] = 1
                objective_matrix[1,state,action] = 1
            
            
            
            pred_by_action = get_pred_by_action(MDPbi,state,action)
            
            for pred in pred_by_action : # pred : (predecesseur,proba,(r1,r2),done)

                first_member_matrix[state,pred[1],action] = first_member_matrix[state,pred[1],action] - (discount_factor * (pred[0][1]))
    
    return first_member_matrix,objective_matrix

infos_state_dict = env.infos_state_dict
nb_states = env.nb_states
nb_actions = 8
"""

mdpbi_exemple = {
    0 : [ [(0,0.75,(0,0),False) , (1,0.25,(0,0),False)] , [ (0,0.5,(0,0),False) , (1,0.5,(0,0),False) ]],
    1 : [ [(0,0.5,(0,0),False) , (1,0.5,(0,0),False)] , [ (0,0.25,(0,0),False) , (1,0.75,(0,0),False) ]]
}

infos_state_dict = { 0 : 0 , 1 : 1}
nb_states = 2
nb_actions = 2

MDPbi = mdpbi_exemple
"""
first_member_matrix,objective_matrix = get_first_member_matrix_and_obj(MDPbi,discount_factor,nb_states,nb_actions,infos_state_dict)

In [30]:
"""
objective_matrix = np.array([
    [[8,12],[11,9]],
    [[13,6],[7,15]]
])
"""
print(objective_matrix)
print("--")
print(first_member_matrix)
# print(first_member_matrix[23])
print("--")
#print(first_member_matrix.reshape((env.nb_states,env.nb_states * 8)))

[[[1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]]]
--
[[[ 0.5      0.5      1.       1.       0.5      0.5      0.5
    0.5    ]
  [ 0.       0.       0.       0.       0.       0.       0.
    0.     ]
  [-0.03125  0.       0.       0.       0.       0.       0.
    0.     ]
  [ 0.       0.       0.       0.       0.       0.       0.
    0.     ]
  [ 0.       0.       0.       0.       0.       0.       0.
    0.     ]
  [ 0.       0.       0.       0.       0.       0.      -0.40625
    0.     ]
  [ 0.       0.       0.       0.      

In [31]:
a = first_member_matrix.reshape((nb_states,nb_states * nb_actions))
b = [0.0 for _ in range(nb_states)]
b[0] = 1.0
c = objective_matrix.reshape((2,nb_states*nb_actions))

print("a shape : ",a.shape)
print("b shape : ",np.array(b).shape)
print("c shape : ",c.shape)

nbcont = nb_states
nbvar = nb_states * nb_actions

lignes = range(nbcont)
colonnes = range(nbvar)

print("nb_cont : ",nbcont)
print("nb_colonnes : ",nbvar)

a shape :  (9, 72)
b shape :  (9,)
c shape :  (2, 72)
nb_cont :  9
nb_colonnes :  72


In [32]:
"""
nbcont=2 
nbvar=4

# Range of plants and warehouses
lignes = range(nbcont)
colonnes = range(nbvar)


# Matrice des contraintes
a = np.array([[5.0/8.0,-1.0/8.0],
[3.0/4.0, -1.0/4.0],
[-1.0/4.0, 3.0/4.0],
[-1.0/8.0, 5.0/8.0]]).T


# Second membre
b = [1.0, 0.0]

# Coefficients de la fonction objectif
c = [[8.0, 12.0, 11.0,9.0],
 [13.0, 6.0, 7.0,15.0]]
"""

'\nnbcont=2 \nnbvar=4\n\n# Range of plants and warehouses\nlignes = range(nbcont)\ncolonnes = range(nbvar)\n\n\n# Matrice des contraintes\na = np.array([[5.0/8.0,-1.0/8.0],\n[3.0/4.0, -1.0/4.0],\n[-1.0/4.0, 3.0/4.0],\n[-1.0/8.0, 5.0/8.0]]).T\n\n\n# Second membre\nb = [1.0, 0.0]\n\n# Coefficients de la fonction objectif\nc = [[8.0, 12.0, 11.0,9.0],\n [13.0, 6.0, 7.0,15.0]]\n'

In [37]:
from gurobipy import *

m = Model("mdp")     
        
# declaration variables de decision
x = []
for i in colonnes:
    x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="x%d" %(i+1)))
    
z = m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="z")

# maj du modele pour integrer les nouvelles variables
m.update()

obj = LinExpr();
obj = z

In [38]:
# definition de l'objectif
m.setObjective(obj,GRB.MAXIMIZE)

l= [1.0 for _ in range(2)]


m.addConstr(quicksum(c[0][j]*x[j] for j in colonnes) >= z, "Controbj 1" )
m.addConstr(quicksum(c[1][j]*x[j] for j in colonnes) >= z, "Controbj 2" )

# Definition des contraintes
for i in lignes:
    m.addConstr(quicksum(a[i][j]*x[j] for j in colonnes) == b[i], "Contrainte %d" % i)
    #print(quicksum(a[i][j]*x[j] for j in colonnes) ,"==",b[i])
    #print("----------------------------------------")
    


m.update()

In [39]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 11 rows, 73 columns and 298 nonzeros
Model fingerprint: 0x5329e1d2
Coefficient statistics:
  Matrix range     [3e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 3 rows and 49 columns
Presolve time: 0.01s
Presolved: 8 rows, 24 columns, 88 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.3000000e+31   1.650000e+31   3.300000e+01      0s
      21    2.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 21 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.000000000e+00


In [40]:
for j in colonnes :
    print(x[j])

<gurobi.Var x1 (value 0.0)>
<gurobi.Var x2 (value 0.0)>
<gurobi.Var x3 (value 0.0)>
<gurobi.Var x4 (value 1.0339534668251207)>
<gurobi.Var x5 (value 0.0)>
<gurobi.Var x6 (value 0.0)>
<gurobi.Var x7 (value 0.0)>
<gurobi.Var x8 (value 0.0)>
<gurobi.Var x9 (value 0.0)>
<gurobi.Var x10 (value 0.1027030401558698)>
<gurobi.Var x11 (value 0.0)>
<gurobi.Var x12 (value 0.0)>
<gurobi.Var x13 (value 0.0)>
<gurobi.Var x14 (value 0.0)>
<gurobi.Var x15 (value 0.0)>
<gurobi.Var x16 (value 0.0)>
<gurobi.Var x17 (value 0.0)>
<gurobi.Var x18 (value 0.05294927493054917)>
<gurobi.Var x19 (value 0.0)>
<gurobi.Var x20 (value 0.0)>
<gurobi.Var x21 (value 0.0)>
<gurobi.Var x22 (value 0.0)>
<gurobi.Var x23 (value 0.0)>
<gurobi.Var x24 (value 0.0)>
<gurobi.Var x25 (value 0.0)>
<gurobi.Var x26 (value 0.0)>
<gurobi.Var x27 (value 0.0)>
<gurobi.Var x28 (value 0.0)>
<gurobi.Var x29 (value 0.0)>
<gurobi.Var x30 (value 0.03600171458030293)>
<gurobi.Var x31 (value 0.0)>
<gurobi.Var x32 (value 0.0)>
<gurobi.Var x33 (va

In [41]:
d = np.array(x).reshape((nb_states,nb_actions))


for i in range(nb_states) :
    for j in range(nb_actions) :
        d[i,j] = d[i,j].x


print(d.sum(axis = 1))   

for i in range(nb_states) :
    d[i,:] /= d.sum(axis = 1)[i]
        
print(d)

[1.0339534668251207 0.1027030401558698 0.05294927493054917
 0.03600171458030293 0.11764705882352941 0.36421375518736543
 0.19240104140851474 0.06616493740888317 0.03396571067986468]
[[0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0]
 [1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0]]
